In [1]:
# use pytorch 0.3.1
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
import crf

import torch
import numpy as np

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path



class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [3]:
from crf import CRF
import utils

importing Jupyter notebook from utils.ipynb


In [4]:
import torch
import numpy as np

In [5]:
# test torch is working
print(os.getcwd())
x = torch.rand(5, 3)
print(x)

/Users/yuxiaozhang/Documents/19spring/cs273/project
tensor([[0.0291, 0.0302, 0.7406],
        [0.2903, 0.8046, 0.6480],
        [0.7522, 0.1258, 0.0423],
        [0.4783, 0.9455, 0.1327],
        [0.0737, 0.5042, 0.0991]])


In [6]:
# two dice one is fair, one is loaded
fair_dice = np.array([1/6]*6)
loaded_dice = np.array([0.04,0.04,0.04,0.04,0.04,0.8])

probabilities = {'fair': fair_dice,
                'loaded': loaded_dice}

In [7]:

# if dice is fair at time t, 0.6 chance we stay fair, 0.4 chance it is loaded at time 2
transition_mat = {'fair': np.array([0.6, 0.4, 0.0]),
                 'loaded': np.array([0.3, 0.7, 0.0]),
                 'start': np.array([0.5, 0.5, 0.0])}
states = ['fair', 'loaded', 'start']
state2ix = {'fair': 0,
           'loaded': 1,
           'start': 2}

# print(fair_dice.reshape(-1,1))
# print(loaded_dice.reshape(-1,1))
# h1 = fair_dice.reshape(-1,1)
# h2 = loaded_dice.reshape(-1,1)
# print(np.hstack([h1, h2]))
log_likelihood = np.hstack([np.log(fair_dice).reshape(-1,1),
                            np.log(loaded_dice).reshape(-1,1)])
print(log_likelihood)

[[-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -0.22314355]]


In [8]:
def simulate_data(n_timesteps):
    data = np.zeros(n_timesteps)
    prev_state = 'start'
    state_list = np.zeros(n_timesteps)
    for n in range(n_timesteps):
        next_state = np.random.choice(states, p=transition_mat[prev_state])
        state_list[n] = state2ix[next_state]
        next_data = np.random.choice([0,1,2,3,4,5], p=probabilities[next_state])
        data[n] = next_data
        prev_state = next_state
    return data, state_list

In [9]:
n_obs = 15
rolls = np.zeros((5000, n_obs)).astype(int)
targets = np.zeros((5000, n_obs)).astype(int)

for i in range(5000):
    data, dices = simulate_data(n_obs)
    rolls[i] = data.reshape(1, -1).astype(int)
    targets[i] = dices.reshape(1, -1).astype(int)

In [10]:
print(log_likelihood)
model = crf.CRF(2, log_likelihood)

[[-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -0.22314355]]
** initializing transition **
** initializing loglikelihood **


/Users/yuxiaozhang/Documents/19spring/cs273/project/crf.py:24: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  self.transition = torch.nn.init.normal(nn.Parameter(torch.randn(n_dice, n_dice + 1)), -1, 0.1)


In [11]:
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam

def crf_train_loop(model, rolls, targets, n_epochs, learning_rate=0.01):
    optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    for epoch in range(n_epochs):
        batch_loss = []
        N = rolls.shape[0]
        model.zero_grad()
        for index, (roll, labels) in enumerate(zip(rolls, targets)):
            # Forward Pass
            neg_log_likelihood = model.neg_log_likelihood(roll, labels)
            batch_loss.append(neg_log_likelihood)
            
            if index % 50 == 0:
                ll = torch.cat(batch_loss).mean()
                ll.backward()
                optimizer.step()
                print("Epoch {}: Batch {}/{} loss is {:.4f}".format(epoch, index//50, N//50, ll.data.numpy()[0]))
                batch_loss = []

    return model

In [12]:
model = crf_train_loop(model, rolls, targets, 1, 0.001)

** inside _data_to_likelihood **
** inside numerator **
self.transition:
 Parameter containing:
tensor([[-1.0393, -0.8443, -0.9535],
        [-1.1236, -1.1344, -1.0686]], requires_grad=True)
self.transition[:, self.n_states]:
 tensor([-0.9535, -1.0686], grad_fn=<SelectBackward>)
loglikelihoods:
 tensor([[-1.7918, -0.2231],
        [-1.7918, -0.2231],
        [-1.7918, -3.2189],
        [-1.7918, -0.2231],
        [-1.7918, -0.2231],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -3.2189],
        [-1.7918, -0.2231],
        [-1.7918, -0.2231]])
loglikelihoods[0].view(1, -1):
 tensor([[-1.7918, -0.2231]])


RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [13]:
print(torch.__version__)

0.4.1
